![banner](https://user-images.githubusercontent.com/5395649/46774810-22395980-ccb9-11e8-8f1a-535769d657ec.png)

# Speech Command Recognition Task with Keras 

In this notebook, we will train a simple convolutional neural network (CNN) to recognize utterances of different words. 

In [ ]:
import keras 
import tensorflow as tf
import numpy as np
import librosa
import os 
import hashlib 
import re
import pandas as pd 
import IPython.display as ipd
from sklearn.metrics import confusion_matrix
from tqdm import tqdm 
import pickle as pkl 
from seaborn import heatmap 

#from preprocess import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)
import tensorflow as tf 
tf.logging.set_verbosity(tf.logging.ERROR)

[librosa](https://librosa.github.io/librosa/) is an interesting python library for audio analysis. The paper describing the library can be found [here](http://conference.scipy.org/proceedings/scipy2015/pdfs/brian_mcfee.pdf).

# Before we get started... 

* **If you are unfamiliar with signal processing in general, we recommend you review the notebook `1-intro-to-audio-data.ipynb`. You need to run this notebook first to download the data to your notebook session**

* You don't need the raw data to run this notebook. You will be able to load a pre-processed dataset that is stored in this repo. This dataset will work for all default parameters provided in this notebook. 

Let us define some variables that will be used throughout the notebook. 

* `data_dir` points to the `wav` folder of the dataset. On the platform, this means `/home/datascience/data/`.
* `MAX_NUM_WAVS_PER_CLASS` is the maximum number of audio clip examples per word class. Leave it as is for now.

## A few words on the dataset 

The dataset we use is the [Speech Commands Dataset](https://arxiv.org/abs/1804.03209). This dataset includes 105k utterances of 35 words. Each word has its corresponding folder containing between 1k and 4k examples. In total, **over 2K different speakers were recorded**. Each clip lasts 1 second at a sampling rate of 16 kHz. The uncompressed data take about 4Gb on disk. 

The dataset is licensed under the [Creative Commons BY 4.0 license](https://creativecommons.org/licenses/by/4.0/). See the LICENSE file in the dataset folder for full details. Its original location was at
[http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz](http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz)

In [ ]:
# the root directory of the dataset. If you are running this notebook on the Platform, this
# folder should be /home/datascience/data/
data_dir = '/home/datascience/block_storage/data/'

MAX_NUM_WAVS_PER_CLASS = 2**27 - 1  # ~134M

# Training, validation and testing data split. An 80-10-10% split is standard. 
training_fraction = 80.0
validation_fraction = 10.0
testing_fraction = 10.0 

# If you have not downloaded the original dataset and you want to 
# use the pre-processed, transformed dataset, set the flag "list_available_words" to False
list_available_words = False 

if list_available_words: 
    # Lets find what words are available for us to classify: 
    words_available = [ d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d)) ] 

    # This is the list of available utterances one can use in their classifier
    print(words_available)

For this particular model, **we will train a classifier whose purpose will be to 
classify utterances of the words "right", "eight", "cat"**

In [ ]:
words_selected = ['right', 'eight', 'cat']

# Build a training, validation and testing sets

In the cells below, we build the training, validation, and training datasets that will be used to train the CNN model. 
We recommend that you load the pre-processed data. It takes a while to compute the MFCCs for these sound clips. **The pre-processed, feature-extracted data was generated for words (`right`, `eight`, `cat`) only.** If you want to use different words, you need to download the raw, original dataset.  

In [ ]:
# Create the mapping between index for one-hot encoding and the class label:
class_to_int = {label:i for i, label in enumerate(words_selected)}
print(class_to_int)

# Save that lookup dictionary to disk: 
pkl.dump(class_to_int, open('./class_label_lookup.pkl','wb'))

In [ ]:
# Either load the processed and feature-extracted dataset or generate your own. 
# Set this value to True unless you have downloaded the raw data locally using the shell script. 

load_preprocessed_data = True

if load_preprocessed_data: 
    df = pkl.load(open('../data/processed_data.pkl','rb'))
else: 
    tqdm.pandas()
    df = pd.DataFrame(columns=['filename','label','sample'])

    for w in words_selected: 
        files = [ f for f in os.listdir(data_dir + w) if f[-4:] == '.wav' ]
        labels = [ w for f in files]
        sample = [ which_set(f, validation_fraction, testing_fraction) for f in files ]
        tmp = pd.DataFrame({'filename':files, 'label':labels, 'sample':sample})
        df = df.append(tmp, ignore_index=True)
   
    # adding the full path, mfcc data and class label mapping 
    df['full_path'] = data_dir + df['label'] + "/" + df['filename']
    df['mfcc'] = df['full_path'].progress_apply(lambda x: data_processing(x))
    df['y_hot'] = df['label'].map(class_to_int)
    
    # save data to disk: 
    df.to_pickle('../data/processed_data.pkl')

In [ ]:
df.head()

In [ ]:
# Let's display the content of the column "mfcc". This column 
# contains the Mel-frequency Cepstral Coefficients (MFCC). 
# It's basically a power spectrum. 

def display_spectrogram(df, id): 
    """Given the index value of the example in the pandas dataframe 
    this function will display a 2-D heatmap of the MFCC spectrum. 
    
    Args: 
      - df (pd.DataFrame) : Dataframe of interest. 
      - id (integer) : row index of the example of interest 
    """
    ax = heatmap(df.iloc[id]['mfcc'], center=-150, cmap="YlGnBu")
    ax.set(xlabel="time chunk", ylabel="mfcc") 

# We will take a look at the first example in the t
display_spectrogram(df, 0)

In [ ]:
# Sanity check: counts for each sample category. 
df['sample'].value_counts()

# Splitting into training and testing sets

Below we split the datasets by training, validation and evaluation (test) samples. 
We will use the mfcc values as the covariates/features for each clip. 

In [ ]:
# Training dataset: 
X_train = df[df['sample'] == "training"]['mfcc'].values
Y_train = df[df['sample'] == "training"]['y_hot'].values

# Validation dataset: 
X_valid = df[df['sample'] == "validation"]['mfcc'].values
Y_valid = df[df['sample'] == "validation"]['y_hot'].values

# Evaluation dataset: 
X_test = df[df['sample'] == "testing"]['mfcc'].values
Y_test = df[df['sample'] == "testing"]['y_hot'].values 

In [ ]:
# Reshaping the MFCCs  
Xtrain = np.asarray([ sub.reshape(20,32,1) for sub in X_train ])
Xvalid = np.asarray([ sub.reshape(20,32,1) for sub in X_valid ])
Xtest = np.asarray([ sub.reshape(20,32,1) for sub in X_test ])

# Converting the index values to categorical values using the Keras to_categorical() function 
Ytrain = to_categorical(Y_train)
Yvalid = to_categorical(Y_valid)
Ytest = to_categorical(Y_test)

In [ ]:
# let's make sure we have the right dimensions for all these arrays: 
Xtrain.shape, Ytrain.shape, Xtest.shape, Ytest.shape

Congratulations! You are now ready to train the Convolutional Neural Network (CNN) model. 

# Training the CNN model 

I adopted a slight variation of the [Sainath & Parada (2015)](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/43969.pdf) `cnn-trad-fpool3` model for the purpose of this demo. I invite the reader to read this paper for more details. 

This is a simple model that performs well. I adopted convolution kernels of equal size in both frequency and time space. I removed the linear low-rank layer. I also used fewer filters and the kernel size is smaller. 

I invite the reader to modify the network architecture and try different approaches. 

In [ ]:
# Convolutional Neural Network (CNN) Model Definition:  

model = Sequential()

# Convolution layer (4x8), 32 filters 
model.add(Conv2D(32, kernel_size=(4, 8), activation='relu', input_shape=(20, 32, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolution layer (2x4), 16 filters 
model.add(Conv2D(16, kernel_size=(2, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
# DNN FC layer: 
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
# softmax layer (3 words)
model.add(Dense(len(words_selected), activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Let's fit the model. 

Ten epochs is good enough to start. It will give you an accuracy superior to 0.90. I invite the reader to train the models for 100+ epochs.

In [ ]:
# Fitting the model: 

model.fit(Xtrain, Ytrain, 
          batch_size=100, 
          epochs=10, 
          verbose=1, 
          shuffle=True,
          validation_data=(Xvalid, Yvalid))

# Model Evaluation

In [ ]:
# Evaluating the model: 

loss, accuracy = model.evaluate(x=Xtest, y=Ytest)
print("loss = {}, accuracy = {}".format(loss, accuracy))

In [ ]:
# Compute and plot Confusion matrix 

Ypred = model.predict_classes(Xtest)
cm = confusion_matrix(Y_test, Ypred, labels=[0,1,2])#, labels = words_selected)

ax = heatmap(cm, annot=True, fmt='d', 
                     linewidths=.2, 
                     xticklabels=words_selected, 
                     yticklabels=words_selected)

ax.set(xlabel='predicted',ylabel='observed')

# Save the model to the Model Catalog using ADS 

In [ ]:
from ads.common.model import ADSModel

keras_ads_model = ADSModel.from_estimator(model, name="Keras")

In [ ]:
keras_ads_model_artifact = keras_ads_model.prepare("/home/datascience/block_storage/keras_ads", force_overwrite=True, 
                                                   fn_artifact_files_included=True, fn_name="predictor")

Here we are going to modify the handler for the Function deployment. We are going to include the following data transformation steps: 

- compute MFCCs
- pad array
- reshape the numpy array to create a spectrogram

behind the Function endpoint. This will allow us to pass a simple waveform when invoking the Function. 

In the cell below we are overwriting `func.py` :  

In [ ]:
%%writefile /home/datascience/block_storage/keras_ads/fn-model/func.py

import io
import json
from json import loads, dumps
import librosa
import pickle as pkl
import numpy as np

# load the lookup dictionary:
words_selected = pkl.load(open('./class_label_lookup.pkl','rb'))
# do a reverse lookup:
words_selected = { value: key for key, value in words_selected.items() }

# Standard :
from fdk import response
import sys
sys.path.append('/function')
import scorefn
model = scorefn.load_model()

def handler(ctx, data: io.BytesIO=None):

    try:
        waveform = json.loads(data.getvalue())['input']
        waveform = np.asarray(waveform)
        max_size = 32
        mfcc = librosa.feature.mfcc(waveform, sr=16000, n_mfcc=20)
        mfcc = np.pad(mfcc, pad_width=((0, 0),(0, max_size-mfcc.shape[1])), mode='constant')
        mfcc = mfcc.reshape(1,20,32,1)
        class_value = model.predict_classes(mfcc)
    except (Exception, ValueError) as ex:
        print(str(ex))

    return response.Response(
        ctx, response_data=json.dumps({"what-you-said-was": words_selected[class_value[0]]}),
        headers={"Content-Type": "application/json"}
    )

In addition, we're going to overwrite the requirements file of `fn-model/` to include ``librosa``: 

In [ ]:
%%writefile /home/datascience/block_storage/keras_ads/fn-model/requirements.txt
keras==2.2.5
tensorflow==1.15
cloudpickle==1.2
pandas==0.24.2
numpy==1.17
scipy==1.3
fdk==0.1.12
scikit-learn==0.21.3
librosa==0.6.2

Lastly, we need to copy over the class label lookup dictionary that we created into the `fn-model/` directory. The Function needs to convert from number to spoken word: 

In [ ]:
%%bash 
cp class_label_lookup.pkl /home/datascience/block_storage/keras_ads/fn-model/

That's it for the Oracle Functions part of the model artifact. We are ready to save to the catalog now! 

## Saving the model artifact to the catalog 

We are now ready to save the artifact to the catalog: 

In [ ]:
compartment_id = os.environ['NB_SESSION_COMPARTMENT_OCID']
project_id = os.environ["PROJECT_OCID"]

# Saving the model artifact to the model catalog: 
model = keras_ads_model_artifact.save(project_id=project_id, compartment_id=compartment_id, 
                                              display_name="Keras Speech Recognition Model 2",
                                              description="Simple speec recognition model built with keras.", 
                                              training_script_path="2-cnn-model-with-keras.ipynb", 
                                              ignore_pending_changes=True)

# What Next?

You can experiment and expand on the model trained in this notebook. Here are a few areas to explore: 

* increase the number of classes. Try to classify 10+ words 
* add noise to your examples. Create a more robust classifier. 
* try different CNN architectures 
* include an "other" class
* include a "silence" class

# References 

* Warden, P. 2018, "Speech Commands: A Dataset for Limited-Vocabulary Speech Recognition", [arXiv:1804.03209](https://arxiv.org/abs/1804.03209)
* Sainath, T.N., Parada, C. 2015 ["Convolutional Neural Networks for Small-footprint Keyword Spotting"]( https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/43969.pdf)

# Appendix

Here's the network architecture for reference. 

![network-architecture](https://user-images.githubusercontent.com/5395649/46776515-cb378280-ccc0-11e8-8dcb-84db4156e981.png)

Below is a function I took directly from the `README` file that came with the dataset. This is a very useful function that ensures testing/validation data does not make its way in the training dataset. We will be using it to assign the sample (training, validation, testing) to which each audio clip belongs. 

In [ ]:
# From Warden (2018) : https://arxiv.org/abs/1804.03209

def which_set(filename, validation_percentage, testing_percentage):
    """Determines which data partition the file should belong to.
    We want to keep files in the same training, validation, or testing sets even
    if new ones are added over time. This makes it less likely that testing
    samples will accidentally be reused in training when long runs are restarted 
    for example. To keep this stability, a hash of the filename is taken and used 
    to determine which set it should belong to. This determination only depends on 
    the name and the set proportions, so it won't change as other files are added.
    
    It's also useful to associate particular files as related (for example words 
    spoken by the same person), so anything after '_nohash_' in a filename is 
    ignored for set determination. This ensures that 'bobby_nohash_0.wav' and 
    'bobby_nohash_1.wav' are always in the same set, for example.
    
    Args
    filename: File path of the data sample.
    validation_percentage: How much of the data set to use for validation.
    testing_percentage: How much of the data set to use for testing.
    
    Returns:
    String, one of 'training', 'validation', or 'testing'.
    """
    base_name = os.path.basename(filename)
    #print(base_name)
    # We want to ignore anything after '_nohash_' in the file name when
    # deciding which set to put a wav in, so the data set creator has a way of
    # grouping wavs that are close variations of each other.
    hash_name = re.sub(r'_nohash_.*$', '', base_name).encode('utf-8')
    # This looks a bit magical, but we need to decide whether this file should
    # go into the training, testing, or validation sets, and we want to keep
    # existing files in the same set even if more files are subsequently
    # added.
    # To do that, we need a stable way of deciding based on just the file name
    # itself, so we do a hash of that and then use that to generate a
    # probability value that we use to assign it. 
    hash_name_hashed = hashlib.sha1(hash_name).hexdigest() 
    percentage_hash = ((int(hash_name_hashed, 16) %
                      (MAX_NUM_WAVS_PER_CLASS + 1)) *
                     (100.0 / MAX_NUM_WAVS_PER_CLASS)) 
    if percentage_hash < validation_percentage:
        result = 'validation'
    elif percentage_hash < (testing_percentage + validation_percentage):
        result = 'testing'
    else:
        result = 'training'
    return result

Below is the data transformation that we apply to each audio clip before feeding the clip to the CNN model. Take a look at the [intro notebook](1-intro-to-audio-data.ipynb) where we discuss spectrogram and MFCCs. 

In [ ]:
def data_processing(filepath, max_len=32):
    """ Compute MFCCs for a given clip
    
    Args: 
        - filepath (str) : path of the wav file to analyze. 
        - max_len (int) : 
        
    Returns: 
        - Mel-frequency cepstral coefficients array. 
    """
    
    # Here we are loading the audio clip using librosa.
    waveform, sampling_rate = librosa.load(filepath, mono=True, sr=None)
    # compute the Mel-frequency cepstral coefficients. n_mfcc 
    # represents the number of coefficients to return. 
    mfcc = librosa.feature.mfcc(waveform, sr=16000, n_mfcc=20)
    
    # Not all clips have the same duration. Padding along the time dimension to 
    # ensure each clip has the same dimensions. Each image fed to the CNN needs 
    # to have the same dimensions. 
    mfcc = np.pad(mfcc, pad_width=((0, 0),(0, max_len-mfcc.shape[1])), mode='constant')
    return mfcc 